In [196]:
!python main.py play --agents q_agent --train 1 --scenario coin-heaven --n-rounds 2000 --no-gui

size of dictionary: 256
  0%|                                          | 2/2000 [00:01<26:16,  1.27it/s]/home/vivi/uni/bomberman/agent_code/q_agent/callbacks.py:232: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  if (neighbours[j] == game_state['bombs'][0]).any():
100%|███████████████████████████████████████| 2000/2000 [12:29<00:00,  2.67it/s]


In [ ]:
!python main.py play --agents q_agent --train 1 --scenario coin-heaven --n-rounds 500 --no-gui

In [197]:
!python main.py play --agents q_agent --scenario coin-heaven --n-rounds 5

size of dictionary: 256
100%|█████████████████████████████████████████████| 5/5 [00:57<00:00, 11.48s/it]


In [198]:
!python main.py play --agents q_agent --scenario clean --n-rounds 3

size of dictionary: 256
  0%|                                                     | 0/3 [00:04<?, ?it/s]


In [ ]:
!python main.py play --my_agent q_agent --scenario classic --save-replay 

In [ ]:
!python main.py replay "replays/replay.pt" --turn-based

In [ ]:
reward = np.loadtxt("agent_code/q_agent/rewards.txt",dtype = float)
plt.plot(reward,'.')
plt.xlabel("rounds")
plt.ylabel("reward")
#plt.ylim(-100,100)

In [3]:
import numpy as np
import itertools

In [201]:
model_trained = np.load("agent_code/q_agent/my-saved-model.npy")

In [213]:
dic = {}
ACTIONS = ['UP', 'RIGHT', 'DOWN', 'LEFT', 'WAIT', 'BOMB']

# rotate clockwise
def get_arrangements(array):
    return np.unique([ np.roll(array, -i) for i in range(0,4) ], axis=0)

# get all possible combinations
def get_variations(array, r=4):
    return np.unique([ item for item in list(itertools.product(array, repeat=r)) ], axis=0)

# compute state to index for the Q-table
def build_state_to_index(arr1 = NEIGHBOURING_FIELDS, arr2 = GAME_MODE, arr3 = CURRENT_FIELD):
    i = 0
    perm = get_variations(arr1)
    comb = np.array(np.meshgrid(arr2, arr3)).T.reshape(-1,2) # combinations of arr2 and arr3
    step = len(comb)
    for p in perm:
        ar = get_arrangements(p)
        for a in ar:
            if tuple(a) not in dic.keys():
                value = [v for v in range(i, i+step)]
                i += step
                dic.update({tuple(a) : value}) 
                
build_state_to_index()


print(np.shape(model_trained))
print(f"# nonzero rows { sum(np.apply_along_axis(np.any, axis=1, arr=model_trained)) }")
print(model_trained[548])

state = [ 0, -1,  2,  2, -1,  2]
print(state)
index = -1
rotation = -1
temp = get_arrangements(state[1:5]) # rotations of neighbouring fields
mode_and_field = [ state[-1], state[0] ]
value_index = 0
comb = np.array(np.meshgrid(GAME_MODE, CURRENT_FIELD)).T.reshape(-1,2)
for i, c in enumerate(comb):
    if (c == mode_and_field).all():
        value_index = i
        print(value_index)
        break
rotation = 0
for i, t in enumerate(temp):
    if (t == state[1:5]).all():
        rotation = i
        print(rotation)
for j, a in enumerate(temp):
    if tuple(a) in dic.keys():
        #print("state: ", state, "index", dic[tuple(a)][value_index])
        index = dic[tuple(a)][value_index]
        print(index)
        break
        
action = np.argmax(model_trained[index]) # Exploit learned values
if action < 4 and rotation != 0: # move and rotated state
    action = (action + rotation) % 4
print(ACTIONS[action])

(3072, 6)
# nonzero rows 17
[-16.67476975  73.67191631   6.26431054   8.93287642   3.55772609
 -19.51158913]
[0, -1, 2, 2, -1, 2]
8
1
548
DOWN


In [40]:
from datetime import datetime

now = datetime.now() 

year = now.strftime("%Y")
print("year:", year)

month = now.strftime("%m")
print("month:", month)

day = now.strftime("%d")
print("day:", day)

time = now.strftime("%H:%M:%S")
print("time:", time)

date_time = now.strftime("%H_%M_%S")
print("date and time:",date_time)

year: 2022
month: 03
day: 09
time: 14:56:48
date and time: 14_56_48


In [55]:
import time
start = time.time()


time.sleep(5)
end  = time.time()
print(end-start)

5.003371000289917
